In [ ]:
import subprocess
import sys
import json
import os
from datetime import datetime

# Install required libraries
try:
    import duckduckgo_search
    import transformers
    import torch
except ImportError:
    print("Installing required packages...")
    # Install all required packages
    subprocess.check_call(
        [sys.executable, "-m", "pip", "install", "-q", 
         "duckduckgo-search", "transformers", "torch"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    print("Libraries installed successfully.")

import pandas as pd
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from transformers import pipeline

# --- CONFIGURATION ---
SILVER_TABLE_NAME = "silver_news"
GOLD_TABLE_NAME = "gold_market_pulse"

try:
    # 1. READ SILVER DATA
    df_silver = spark.read.table(SILVER_TABLE_NAME)
    
    # 2. DEFINE THE AI MODEL (Pandas UDF)
    @pandas_udf(StructType([
        StructField("label", StringType()),
        StructField("score", FloatType())
    ]))
    def sentiment_analysis_udf(text_series: pd.Series) -> pd.DataFrame:
        # Load model inside function for distributed execution
        pipe = pipeline(
            "sentiment-analysis", 
            model="distilbert-base-uncased-finetuned-sst-2-english"
        )
        
        # Run sentiment analysis on batch
        results = pipe(text_series.tolist(), truncation=True, max_length=512)
        
        # Convert to DataFrame with correct schema
        return pd.DataFrame(results)
    
    # 3. APPLY THE MODEL
    df_scored = df_silver.withColumn(
        "sentiment_result", 
        sentiment_analysis_udf(col("snippet"))
    )
    
    # 4. FLATTEN RESULTS
    df_gold = df_scored.select(
        col("date"),
        col("competitor_tag"),
        col("source"),
        col("title"),
        col("url"),
        col("snippet"),
        col("sentiment_result.label").alias("sentiment_label"),
        col("sentiment_result.score").alias("confidence_score")
    )
    
    # 5. SAVE TO GOLD
    df_gold.write.mode("overwrite").format("delta").saveAsTable(GOLD_TABLE_NAME)
    
    # Prepare pipeline output
    record_count = df_gold.count()
    result = {
        "status": "success",
        "records_analyzed": record_count,
        "table_name": GOLD_TABLE_NAME,
        "timestamp": datetime.now().isoformat()
    }
    
    # Exit with JSON for pipeline
    mssparkutils.notebook.exit(json.dumps(result))

except Exception as e:
    # Handle errors gracefully
    error_result = {
        "status": "failed",
        "error": str(e),
        "timestamp": datetime.now().isoformat()
    }
    mssparkutils.notebook.exit(json.dumps(error_result))

In [ ]:
%%sql
SELECT 
    date, 
    competitor_tag, 
    sentiment_label, 
    confidence_score, 
    snippet 
FROM gold_market_pulse 
ORDER BY date DESC 
LIMIT 10